<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=71ee0b551ff79eaa3438ec2374b7c604a781fcfcf939994f2688701eac9e8063
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-02 11:49:38
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14


-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 60.53 K (0.42%)
Current PnL: -24.96 L (-15.91%)
CY Booked + Current PnL: -10.67 L (-6.8%)
-------------------
Total profit:  1.88 L
Total loss:  -26.84 L
-------------------
Total Booked + Current PnL: 16.31 L (12.57%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.92%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.92 L (62.39%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,3.65,12.24,6.64,19.69,14291.0,23478.0,215220.0,1.27,63.0,X-MC,2.48,78.0,1.64,1.49,19.87,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.19,-20.20,25.43,0.09,20451.0,-20356.0,80421.0,86.77,37.0,M-SC,3.49,253.0,-1.00,0.56,5.66,OX40N,NTT,DURABLES
51,NESTLEIND,1377.00,-0.78,13.28,7.17,21.40,21558.0,35241.0,300667.0,5.06,62.0,X-LC,5.07,12.0,1.63,2.09,20.44,XY25,NTT,FMCG
49,MASFIN,398.61,0.44,-1.71,24.18,22.05,23285.0,-1680.0,96300.0,-15.09,54.0,H-SC,6.45,164.0,-0.07,0.67,39.98,XR,ATH,FINANCE
17,COALINDIA,484.83,3.20,1.62,17.32,19.23,26483.0,2442.0,152902.0,19.88,77.0,L-LC,4.44,179.0,0.09,1.06,23.24,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,DABUR,735.00,4.32,1.97,40.93,43.70,103533.0,4884.0,252952.0,-6.50,68.0,X-MC,1.91,72.0,0.05,1.76,17.32,XY24,BTT,FMCG
57,RECLTD,446.00,4.03,3.55,16.60,20.74,34922.0,7205.0,210375.0,35.83,73.0,M-MC,7.86,189.0,0.21,1.46,14.54,XY25,NTT,FINANCE
82,VOLTAS,1530.00,3.65,12.24,6.64,19.69,14291.0,23478.0,215220.0,1.27,63.0,X-MC,2.48,78.0,1.64,1.49,19.87,XY25,NTT,AC
80,VALIANTORG,838.00,3.43,-41.72,198.81,74.15,189571.0,-68252.0,95353.0,-271.55,56.0,H-SC,21.09,149.0,-0.36,0.66,22.23,XR,NTT,CHEMICALS
17,COALINDIA,484.83,3.20,1.62,17.32,19.23,26483.0,2442.0,152902.0,19.88,77.0,L-LC,4.44,179.0,0.09,1.06,23.24,XY25,ATH,MINING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-3.89,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
0,5PAISA,593.00,-2.44,-30.19,61.73,12.90,94154.0,-65970.0,152526.0,124.76,67.0,H-SC,11.48,161.0,-0.70,1.06,26.82,OX40N,NTT,FINANCE
39,INDIGOPNTS,1408.00,-1.70,-17.11,20.67,0.02,29906.0,-29876.0,144683.0,120.42,46.0,M-SC,6.77,234.0,-1.00,1.00,25.85,OX40N,NTT,PAINTS
9,BAJAJHLDNG,14451.52,-1.69,-0.30,29.61,29.22,56126.0,-575.0,189550.0,-2.67,46.0,X-LC,12.10,35.0,-0.01,1.32,8.85,X40,ATH,FINANCE
12,BERGEPAINT,680.00,-1.58,-6.44,29.47,21.14,62684.0,-14633.0,212706.0,-13.02,39.0,X-MC,2.03,74.0,-0.23,1.48,18.12,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.36,-0.76,109.5,107.91,160444.0,-1120.0,146524.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.25,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.36,-0.76,109.50,107.91,160444.0,-1120.0,146524.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.25,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.70,-17.11,20.67,0.02,29906.0,-29876.0,144683.0,120.42,46.0,M-SC,6.77,234.0,-1.00,1.00,25.85,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.19,-20.20,25.43,0.09,20451.0,-20356.0,80421.0,86.77,37.0,M-SC,3.49,253.0,-1.00,0.56,5.66,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.00,-20.79,43.26,13.47,92402.0,-56070.0,213597.0,-67.53,56.0,H-SC,6.11,158.0,-0.61,1.48,8.61,XY24,NTT,PAINTS
66,SIS,528.00,1.84,-22.48,56.42,21.26,48564.0,-24957.0,86075.0,2032.64,57.0,H-SC,4.62,166.0,-0.51,0.60,16.30,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,4.03,3.55,16.60,20.74,34922.0,7205.0,210375.0,35.83,73.0,M-MC,7.86,189.0,0.21,1.46,14.54,XY25,NTT,FINANCE
17,COALINDIA,484.83,3.20,1.62,17.32,19.23,26483.0,2442.0,152902.0,19.88,77.0,L-LC,4.44,179.0,0.09,1.06,23.24,XY25,ATH,MINING
84,WIPRO,420.00,1.68,11.50,54.50,72.27,102970.0,19488.0,188936.0,-4.26,68.0,M-LC,5.68,99.0,0.19,1.31,18.39,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.68,11.50,54.50,72.27,102970.0,19488.0,188936.0,-4.26,68.0,M-LC,5.68,99.0,0.19,1.31,18.39,XR,NTT,IT
25,FINCABLES,1641.55,-0.36,-0.76,109.50,107.91,160444.0,-1120.0,146524.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.25,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.43,-7.06,122.43,106.72,140339.0,-8708.0,114628.0,-54.06,35.0,H-SC,6.49,139.0,-0.06,0.80,14.67,AR,ATH,MISC
49,MASFIN,398.61,0.44,-1.71,24.18,22.05,23285.0,-1680.0,96300.0,-15.09,54.0,H-SC,6.45,164.0,-0.07,0.67,39.98,XR,ATH,FINANCE
85,ZYDUSLIFE,1286.17,0.15,-3.36,40.34,35.63,81703.0,-7039.0,202535.0,-17.18,45.0,H-MC,2.90,119.0,-0.09,1.41,12.75,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-3.89,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
22,DIXON,18931.72,0.88,-15.43,55.20,31.26,107733.0,-35600.0,195168.0,-60.51,33.0,X-MC,17.04,56.0,-0.33,1.35,3.66,X40N,ATH,IT
23,DMART,5391.45,0.29,-10.41,44.67,29.60,99886.0,-25992.0,223608.0,-23.56,34.0,X-LC,9.15,19.0,-0.26,1.55,9.48,X40N,ATH,FMCG
7,AWL,485.00,-0.52,-22.80,103.61,57.19,260866.0,-74350.0,251777.0,-63.50,37.0,X-MC,7.34,58.0,-0.29,1.75,1.04,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.03,-38.95,121.46,35.20,95413.0,-50115.0,78555.0,2.96,38.0,X-SC,17.35,92.0,-0.53,0.55,0.90,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-0.94,-17.73,52.56,25.51,80472.0,-32990.0,153105.0,-6.04,41.0,H-LC,1.20,49.0,-0.41,1.06,12.83,AR,ATH,ELECTRICAL
78,UNITDSPR,1644.0,-1.08,2.03,18.36,20.77,43864.0,4764.0,238908.0,-6.85,41.0,X-MC,1.30,62.0,0.11,1.66,9.09,X40N,NTT,BREWERIES
20,DABUR,735.0,4.32,1.97,40.93,43.70,103533.0,4884.0,252952.0,-6.50,68.0,X-MC,1.91,72.0,0.05,1.76,17.32,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-1.58,-6.44,29.47,21.14,62684.0,-14633.0,212706.0,-13.02,39.0,X-MC,2.03,74.0,-0.23,1.48,18.12,XY24,NTT,PAINTS
82,VOLTAS,1530.0,3.65,12.24,6.64,19.69,14291.0,23478.0,215220.0,1.27,63.0,X-MC,2.48,78.0,1.64,1.49,19.87,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-3.89,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
11,BATAINDIA,2096.00,-0.03,-38.95,121.46,35.20,95413.0,-50115.0,78555.0,2.96,38.0,X-SC,17.35,92.0,-0.53,0.55,0.90,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.15,-26.75,124.10,64.15,216306.0,-63651.0,174300.0,-57.19,38.0,X-SC,5.90,82.0,-0.29,1.21,1.03,XY24,BTT,CEMENT
7,AWL,485.00,-0.52,-22.80,103.61,57.19,260866.0,-74350.0,251777.0,-63.50,37.0,X-MC,7.34,58.0,-0.29,1.75,1.04,XY24,NTT,FMCG
52,PAGEIND,51605.07,0.84,-9.02,43.57,30.62,93967.0,-21384.0,215670.0,-35.36,38.0,X-MC,15.47,55.0,-0.23,1.50,1.07,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,2.19,-28.28,99.16,42.84,46231.0,-18383.0,46623.0,-53.17,51.0,X-SC,30.49,83.0,-0.40,0.32,5.48,XY24,NTT,MISC
58,RELAXO,1176.00,0.53,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,47.0,X-SC,7.46,91.0,-0.48,0.53,1.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.03,-38.95,121.46,35.20,95413.0,-50115.0,78555.0,2.96,38.0,X-SC,17.35,92.0,-0.53,0.55,0.90,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.46,-0.48,24.67,24.08,31168.0,-609.0,126341.0,-8.54,52.0,X-SC,7.63,89.0,-0.02,0.88,19.04,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.15,-17.90,74.57,43.32,99715.0,-29152.0,133720.0,-29.42,41.0,X-SC,7.69,90.0,-0.29,0.93,2.67,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.40,-10.90,35.48,20.71,119564.0,-41246.0,336991.0,-23.30,53.0,X-LC,2.81,1.0,-0.34,2.34,12.60,X40,ATH,IT
41,INFY,1972.00,0.69,11.72,20.16,34.24,70470.0,36666.0,349554.0,-13.14,58.0,X-LC,7.43,2.0,0.52,2.43,20.97,X40,NTT,IT
75,TMPV,600.00,1.06,-23.87,61.53,22.98,105820.0,-53917.0,171981.0,-20.63,63.0,X-LC,5.49,3.0,-0.51,1.19,8.17,XY24,NTT,AUTO
81,VBL,671.64,-0.08,-0.75,36.69,35.67,115016.0,-2361.0,313481.0,-12.49,63.0,X-LC,2.68,4.0,-0.02,2.18,13.00,X40N,ATH,FMCG
43,ITC,452.00,-3.89,-14.29,29.25,10.78,60247.0,-34345.0,205973.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.11,-32.89,98.02,32.89,50787.0,-25394.0,51813.0,-739.67,72.0,L-MC,6.70,259.0,-0.50,0.36,42.69,XR,NTT,BANKS
49,MASFIN,398.61,0.44,-1.71,24.18,22.05,23285.0,-1680.0,96300.0,-15.09,54.0,H-SC,6.45,164.0,-0.07,0.67,39.98,XR,ATH,FINANCE
13,BSOFT,831.70,-0.19,-21.82,92.88,50.79,102127.0,-30697.0,109956.0,0.12,53.0,H-SC,10.80,151.0,-0.30,0.76,28.02,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.70,-17.11,20.67,0.02,29906.0,-29876.0,144683.0,120.42,46.0,M-SC,6.77,234.0,-1.00,1.00,25.85,OX40N,NTT,PAINTS
69,SURYODAY,216.00,1.02,-18.49,50.94,23.03,74353.0,-33109.0,145962.0,60.09,57.0,H-SC,5.69,167.0,-0.45,1.01,44.82,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,ICICIPRULI,790.00,0.79,12.46,16.24,30.73,35980.0,24545.0,221550.0,-13.81,74.0,X-MC,5.05,75.0,0.68,1.54,26.82,X40,ATH,INSURANCE
74,TITAGARH,1548.00,0.69,-16.41,74.35,45.74,150507.0,-39747.0,202430.0,11.43,62.0,H-SC,3.97,174.0,-0.26,1.40,31.66,XY24,NTT,ENGINEERING
0,5PAISA,593.00,-2.44,-30.19,61.73,12.90,94154.0,-65970.0,152526.0,124.76,67.0,H-SC,11.48,161.0,-0.70,1.06,26.82,OX40N,NTT,FINANCE
17,COALINDIA,484.83,3.20,1.62,17.32,19.23,26483.0,2442.0,152902.0,19.88,77.0,L-LC,4.44,179.0,0.09,1.06,23.24,XY25,ATH,MINING
84,WIPRO,420.00,1.68,11.50,54.50,72.27,102970.0,19488.0,188936.0,-4.26,68.0,M-LC,5.68,99.0,0.19,1.31,18.39,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.11
1,25,44.13
2,50,76.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.86
MC    30.88
LC    26.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.78
X40      23.82
X40N     12.80
XY25     10.60
XR        9.00
AR        8.56
OX40N     7.49
SR        0.98
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
X-LC    22.86
H-SC    22.82
M-SC    11.38
X-SC     7.85
H-MC     4.68
M-MC     1.46
M-LC     1.31
L-LC     1.06
H-LC     1.06
L-SC     0.81
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.98,-7.73,43.07
IT,12.82,-15.85,74.29
FINANCE,11.09,-11.68,58.55
MISC,6.99,-28.96,81.71
ELECTRICAL,5.81,-12.08,53.14
PAINTS,5.65,-13.33,30.16
INSURANCE,4.77,1.24,32.56
PHARMA,3.94,-3.55,36.15
AUTO,2.87,-26.75,70.69


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3111054.0,21
AR,1341586.0,10
XR,1192622.0,12
X40,1090779.0,15
X40N,898307.0,9
OX40N,712586.0,10
XY25,365971.0,7
SR,279379.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3506295.0,24
X-MC,1473535.0,16
M-SC,1418977.0,15
X-LC,931998.0,13
X-SC,788182.0,8
H-MC,404689.0,3
L-SC,172974.0,2
M-LC,102970.0,1
H-LC,80472.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1217845.0      6
           AR         918516.0      5
M-SC       XY24       802973.0      6
H-SC       XR         776327.0      7
X-MC       X40        494490.0      7
           XY24       427083.0      3
X-LC       X40        411592.0      6
X-MC       X40N       373625.0      4
H-SC       OX40N      314228.0      4
X-SC       X40N       309780.0      3
M-SC       OX40N      309291.0      5
X-SC       XY24       293705.0      3
H-SC       SR         279379.0      2
X-LC       X40N       214902.0      2
H-MC       AR         214516.0      2
           XY24       190173.0      1
X-SC       X40        184697.0      2
X-LC       XY24       179275.0      2
M-SC       XR         178631.0      2
X-MC       XY25       178337.0      2
M-SC       AR         128082.0      2
X-LC       XY25       126229.0      3
M-LC       XR         102970.0      1
L-SC       OX40N       89067.0      1
           XR          83907.0      1
H-LC       AR          80472.0      1
L-MC       XR          50787.0      1
M-MC       XY25        34922.0      1
L-LC       XY25        26483.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 59.0 seconds
